# Demo 01 - Introduction to .NET Interactive and F# #

This notebook showcases an F# kernel running on Jupyter Notebooks using .NET Interactive

F# is an open-source, cross-platform functional first programming language for .NET. Note that the contents of this notebook are not an exhaustive list of the F# language features.

## Operations

Arithmentic operations can be written and evaluated inline.

In [1]:
(2 + 4) / 3

2

## Variables

Store values in variables using the `let` keyword. By default, data is immutable in F#

In [2]:
let x = 1

In [3]:
x

1

Use the `immutable` keyword to make your data mutable.

In [4]:
let mutable y = 3

In [5]:
y

3

To assign a new value to a mutable variable, use the `<-` operator.

In [6]:
y <- 5

In [7]:
y

5

## Functions

Functions can be defined with the following syntax:

```
let [function-name] <params> = 
    {function-body}
```

F# is whitespace delimited (no curly braces).

The last line in the function contains the return value or expression (if any). However, there's no need to use the `return` keyword.

In [8]:
let add x y =
    x + y

In [9]:
add 1 2

3

### Partially applied functions

With partially applied functions, you can supply some of the parameters of a function to create a new function that expects the remaining parameters.

In [10]:
let addOne = add 1

Then, you can use the partially applied function just like any other function and supply the remaining parameters to get the results.

In [11]:
addOne 3

4

An alternative way to apply functions is using the pipe `|>` operator. The pipe operator uses the expression on the left-hand side and uses it as input for the expression in the right-hand side.

In [12]:
3 |> addOne

4

The pipe operatator can be extremely useful when applying several expressions to a piece of data.

In [13]:
let multiplyByTwo x = 
    x * 2

In [14]:
3 |> addOne |> multiplyByTwo

8

## Collections

F# supports various types of collections which include:

| Type | Description |
| --- | --- |
| List | An ordered, immutable series of elements of the same type. Implemented as a linked list.|
| Array | A fixed-size, zero-based, mutable collection of consecutive data elements that are all of the same type. |
| Sequence (IEnumerable) | A logical series of elements that are all of one type. Sequences are represented by the `seq<'T>` type, which is an alias for `IEnumerable<T>`. Therefore, any .NET Framework type that implements `System.Collections.Generic.IEnumerable<'T>` can be used as a sequence. |
| Set | An immutable set that's based on binary trees, where comparison is the F# structural comparison function, which potentially uses implementations of the `System.IComparable` interface on key values.|
| Map | An immutable dictionary of elements. Elements are accessed by key.|

<sub>Source: Microsoft Docs</sub>

The code below creates a list of numbers from 0 to 10 and skips every other value

In [15]:
let numbers = [0..2..10]

In [16]:
numbers

index,value
0,0
1,2
2,4
3,6
4,8
5,10


You can also use the pipe `|>` operator in conjunction to built-in collection operations to apply functions to the values in a collection.

In [17]:
let transformedNumbers = 
    numbers
    |> List.map addOne
    |> List.map multiplyByTwo

In [18]:
transformedNumbers

index,value
0,2
1,6
2,10
3,14
4,18
5,22


Alternatively, you can use function composition as well.

In [19]:
let addOneAndMultiplyByTwo = addOne >> multiplyByTwo

In [20]:
numbers |> List.map addOneAndMultiplyByTwo

index,value
0,2
1,6
2,10
3,14
4,18
5,22


## Plotting

Using `#r` you can reference and install packages to use in your notebook. In this case, the [XPlot.Plotly](https://www.nuget.org/packages/XPlot.Plotly/) package is used to plot data points.

In [21]:
#r "nuget:XPlot.Plotly"

In [22]:
open XPlot.Plotly

Installed package XPlot.Plotly version *

Unhandled exception: System.ArgumentNullException: Value cannot be null. (Parameter 'directory')
   at Microsoft.DotNet.Interactive.CompositeKernelExtensionLoader.LoadFromDirectoryAsync(DirectoryInfo directory, CompositeKernel kernel, KernelInvocationContext context)
   at Microsoft.DotNet.Interactive.CompositeKernel.LoadExtensionsFromDirectory(DirectoryInfo directory, KernelInvocationContext context)
   at Microsoft.DotNet.Interactive.Commands.LoadExtensionsInDirectory.InvokeAsync(KernelInvocationContext context)
   at Microsoft.DotNet.Interactive.CompositeKernel.HandleAsync(IKernelCommand command, KernelInvocationContext context)
   at Microsoft.DotNet.Interactive.KernelCommandPipeline.<BuildPipeline>b__6_0(IKernelCommand command, KernelInvocationContext context, KernelPipelineContinuation _)
   at Microsoft.DotNet.Interactive.KernelBase.<AddSetKernelMiddleware>b__9_0(IKernelCommand command, KernelInvocationContext context, KernelPipelineContinuation next)
   at Microsoft.DotN

In [23]:
transformedNumbers
|> Chart.Line

In [24]:
[
    numbers |> List.mapi(fun i x -> i,x)
    transformedNumbers |> List.mapi(fun i x -> i,x)
]
|> Chart.Line
|> Chart.WithTitle "Comparison"
|> Chart.WithLegend true
|> Chart.WithHeight 400
|> Chart.WithWidth 600
|> Chart.WithLabels ["Original";"Transformed"]

Sometimes, you may want to perform an operation on the list for which there is no existing / defined function. In that case, you can use a lambda expression by using the `fun` keyword. The operation below uses the built-in `filter` operation along with a lambda expression to return only numbers divisible by 3. 

In [25]:
let divisibleByThree = 
    transformedNumbers
    |> List.filter(fun num -> num % 3 = 0)

In [26]:
divisibleByThree

index,value
0,6
1,18


## Types

There are several ways to define types in F#. 

### Records

One way is to use Records. Records represent simple aggregates of named values, optionally with members. They can either be structs or reference types. They are reference types by default. 

In [27]:
type Person = {
    FirstName: string
    LastName: string
}

In [28]:
let luis = {FirstName="Luis"; LastName="Quintanilla"}

In [29]:
printfn "Hello %s" luis.FirstName

Hello Luis


### Classes

Another way to define types is using classes.

Classes are types that represent objects that can have properties, methods, and events.

In [30]:
type Person (firstName:string, lastName:string) as self = 
    member self.firstName = firstName
    member self.lastName = lastName

In [31]:
let alice = Person("Alice","Doe")

In [32]:
printfn "Hello %s" alice.firstName

Hello Alice


### Discriminated Union (DU)

Discriminated unions provide support for values that can be one of a number of named cases, possibly each with different values and types.

In [33]:
type Employee = 
    | IndividualContributor of id:int * title:string
    | Manager of id:int * title:string * directReports: Employee list

In [34]:
let dunderMifflin = [
    Manager(id=1, title="Branch Manager", directReports = [
        Manager(id=2,title="Assistant to the regional manager",directReports=[
            IndividualContributor(id=6,title="Beet Farmer")
        ])
        IndividualContributor(id=3,title="Receptionist")
        IndividualContributor(id=5,title="Accountant")
    ])
    IndividualContributor(id=4,title="Quality Assurance Manager")
]

See [Microsoft Docs](https://docs.microsoft.com/dotnet/fsharp/language-reference/records#differences-between-records-and-classes) for more information on types.